In [1]:
pip install simplekml


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for simplekml: filename=simplekml-1.3.6-py3-none-any.whl size=65874 sha256=940ceab6cef1974af7ab817df25b6a782fb4cf30e86974f706f145ef1b02a3b5
  Stored in directory: c:\users\ags_asc\appdata\local\pip\cache\wheels\83\ee\f2\65cecfd948f1429ead035fd6d56bc6bd6574a636ddc4d65cbd
Successfully built simplekml
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import zipfile
import simplekml
from xml.etree import ElementTree as ET

# Giriş və çıxış yollarını təyin edin Point KMZ
input_folder = r"C:\Users\AGS_ASC\Desktop\KOORDINATLAR"  # Burada qovluğun tam yolunu qeyd edin
output_kmz = r"C:\Users\AGS_ASC\Desktop\test4\merged.kmz"  # Çıxış faylının adı və yolu

# Yeni KML faylı yaradılır
merged_kml = simplekml.Kml()

def add_kml_content(kml_string, merged_kml):
    """Mövcud KML məzmununu yeni KML-ə əlavə edir"""
    try:
        root = ET.fromstring(kml_string)  # XML strukturunu oxuyur
        for placemark in root.findall(".//{http://www.opengis.net/kml/2.2}Placemark"):
            name = placemark.find("{http://www.opengis.net/kml/2.2}name")
            point = placemark.find(".//{http://www.opengis.net/kml/2.2}coordinates")

            if point is not None:
                coords = point.text.strip().split(",")
                lon, lat = float(coords[0]), float(coords[1])
                pm = merged_kml.newpoint(name=name.text if name is not None else "Unnamed", coords=[(lon, lat)])
                pm.style.iconstyle.icon.href = "http://maps.google.com/mapfiles/kml/pushpin/red-pushpin.png"
    except Exception as e:
        print(f"KML formatında xəta: {e}")

# Qovluqdakı bütün KML və KMZ fayllarını oxumaq
for file in os.listdir(input_folder):
    file_path = os.path.join(input_folder, file)

    if file.endswith(".kml"):
        with open(file_path, "r", encoding="utf-8") as f:
            kml_content = f.read()
            add_kml_content(kml_content, merged_kml)

    elif file.endswith(".kmz"):
        with zipfile.ZipFile(file_path, "r") as kmz:
            for kmz_file in kmz.namelist():
                if kmz_file.endswith(".kml"):
                    with kmz.open(kmz_file) as f:
                        kml_content = f.read().decode("utf-8")
                        add_kml_content(kml_content, merged_kml)

# Yaradılmış KML-ni KMZ formatına çeviririk
kml_file = output_kmz.replace(".kmz", ".kml")
merged_kml.save(kml_file)

# KMZ faylına çevirmək
with zipfile.ZipFile(output_kmz, "w", zipfile.ZIP_DEFLATED) as kmz:
    kmz.write(kml_file, os.path.basename(kml_file))

# Ara faylı silirik
os.remove(kml_file)

print("Bütün KML və KMZ faylları birləşdirildi və 'merged.kmz' kimi yadda saxlandı.")


KML formatında xəta: could not convert string to float: ''
Bütün KML və KMZ faylları birləşdirildi və 'merged.kmz' kimi yadda saxlandı.


In [3]:
import os
import zipfile
import simplekml
from xml.etree import ElementTree as ET

# Giriş və çıxış yollarını təyin edin Polygon kmz
input_folder = r"C:\Users\AGS_ASC\Desktop\KOORDINATLAR"  # Burada qovluğun tam yolunu qeyd edin
output_kmz = r"C:\Users\AGS_ASC\Desktop\test4\merged_2.kmz"  # Çıxış faylının adı və yolu

# Yeni KML faylı yaradılır
merged_kml = simplekml.Kml()

def add_polygon_to_kml(kml_string, merged_kml):
    """Mövcud KML məzmunundakı poliqonları yeni KML-ə əlavə edir"""
    try:
        root = ET.fromstring(kml_string)  # XML formatını oxuyur
        for placemark in root.findall(".//{http://www.opengis.net/kml/2.2}Placemark"):
            name = placemark.find("{http://www.opengis.net/kml/2.2}name")
            polygon = placemark.find(".//{http://www.opengis.net/kml/2.2}coordinates")

            if polygon is not None:
                coords_text = polygon.text.strip().split()
                coords = [(float(c.split(",")[0]), float(c.split(",")[1])) for c in coords_text]
                
                poly = merged_kml.newpolygon(name=name.text if name is not None else "Unnamed Polygon",
                                             outerboundaryis=coords)
                
                # Poliqlonun tərzini (style) müəyyən edirik
                poly.style.polystyle.color = simplekml.Color.changealpha("7F", simplekml.Color.red)  # Qırmızı şəffaf
                poly.style.linestyle.color = simplekml.Color.red  # Kənar xətt qırmızı olsun
    except Exception as e:
        print(f"KML formatında xəta: {e}")

# Qovluqdakı bütün KML və KMZ fayllarını oxumaq
for file in os.listdir(input_folder):
    file_path = os.path.join(input_folder, file)

    if file.endswith(".kml"):
        with open(file_path, "r", encoding="utf-8") as f:
            kml_content = f.read()
            add_polygon_to_kml(kml_content, merged_kml)

    elif file.endswith(".kmz"):
        with zipfile.ZipFile(file_path, "r") as kmz:
            for kmz_file in kmz.namelist():
                if kmz_file.endswith(".kml"):
                    with kmz.open(kmz_file) as f:
                        kml_content = f.read().decode("utf-8")
                        add_polygon_to_kml(kml_content, merged_kml)

# Yaradılmış KML-ni KMZ formatına çeviririk
kml_file = output_kmz.replace(".kmz", ".kml")
merged_kml.save(kml_file)

# KMZ faylına çevirmək
with zipfile.ZipFile(output_kmz, "w", zipfile.ZIP_DEFLATED) as kmz:
    kmz.write(kml_file, os.path.basename(kml_file))

# Ara faylı silirik
os.remove(kml_file)

print("Bütün KML və KMZ faylları birləşdirildi və 'merged.kmz' kimi yadda saxlandı.")


Bütün KML və KMZ faylları birləşdirildi və 'merged.kmz' kimi yadda saxlandı.
